⊕ [结构化流编程指南 - Spark 2.4.0文档](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)


## 概观
结构化流是一种基于Spark SQL引擎的可扩展且容错的流处理引擎。您可以像表达静态数据的批处理计算一样表达流式计算。Spark SQL引擎将负责逐步和连续地运行它，并在流数据继续到达时更新最终结果。您可以使用Scala，Java，Python或R中的数据集/数据框架API来表示流聚合，事件时间窗口，流到批处理连接等。计算在同一优化的Spark SQL引擎上执行。最后，系统通过检查点和预写日志确保端到端的一次性容错保证。简而言之，结构化流传输提供快速，可扩展，容错，端到端的精确一次流处理，而无需用户推理流式传输。

在内部，默认情况下，结构化流式查询使用微批处理引擎进行处理，该引擎将数据流作为一系列小批量作业处理，从而实现低至100毫秒的端到端延迟和完全一次的容错保证。但是，自Spark 2.3以来，我们引入了一种称为连续处理的新型低延迟处理模式，它可以实现低至1毫秒的端到端延迟，并且具有至少一次保证。无需更改查询中的数据集/数据框操作，您就可以根据应用程序要求选择模式。

在本指南中，我们将引导您完成编程模型和API。我们将解释大多使用默认的微批处理模式的概念，再后来讨论连续处理模式。首先，让我们从一个结构化流式查询的简单示例开始 - 一个流式字数。

In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder
  .appName("StructuredNetworkWordCount")
  .getOrCreate()
  
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@217f98d5


org.apache.spark.sql.SparkSession@217f98d5

In [2]:
// Create DataFrame representing the stream of input lines from connection to localhost:9999
val lines = spark.readStream
  .format("socket")
  .option("host", "localhost")
  .option("port", 9999)
  .load()

// Split the lines into words
val words = lines.as[String].flatMap(_.split(" "))

// Generate running word count
val wordCounts = words.groupBy("value").count()

lines = [value: string]
words = [value: string]
wordCounts = [value: string, count: bigint]


[value: string, count: bigint]

此linesDataFrame表示包含流文本数据的无界表。此表包含一列名为“value”的字符串，并且流式文本数据中的每一行都成为表中的一行。请注意，由于我们只是设置转换，并且尚未启动它，因此目前没有接收任何数据。接下来，我们使用了将DataFrame转换为String的数据集.as[String]，以便我们可以应用flatMap操作将每行拆分为多个单词。结果words数据集包含所有单词。最后，我们wordCounts通过对数据集中的唯一值进行分组并对其进行计数来定义DataFrame。请注意，这是一个流式DataFrame，它表示流的运行字数。

我们现在已经设置了关于流数据的查询。剩下的就是实际开始接收数据并计算计数。为此，我们将其设置为outputMode("complete")每次更新时将完整的计数集（指定者）打印到控制台。然后使用启动流式计算start()。



In [3]:
// Start running the query that prints the running counts to the console
val query = wordCounts.writeStream
  .outputMode("complete")
  .format("console")
  .start()

query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
+-----+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
|hello|    1|
|world|    1|
+-----+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
|  boy|    1|
|  you|    1|
|hello|    1|
|  bad|    1|
|  are|    1|
|world|    1|
+-----+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+-------+-----+
|  value|count|
+-------+-----+
|    boy|    1|
|    you|    1|
|  hello|    1|
|    bad|    1|
|student|    1|
|      i|    1|
|    are|    1|
|  world|    1|
|      a|    1|
|     am|    1|
+-------+-----+



Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:998)
  at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1304)
  at java.util.concurrent.CountDownLatch.await(CountDownLatch.java:231)
  at org.apache.spark.sql.execution.streaming.StreamExecution.awaitTermination(StreamExecution.scala:467)
  at org.apache.spark.sql.execution.streaming.StreamingQueryWrapper.awaitTermination(StreamingQueryWrapper.scala:53)

执行此代码后，流式计算将在后台启动。该query对象是该活动流式查询的句柄，我们决定等待查询终止，awaitTermination()以防止在查询处于活动状态时退出该进程。

要实际执行此示例代码，您可以在自己的Spark应用程序中编译代码 ，或者只需 在下载Spark后运行该示例。我们正在展示后者。您首先需要使用Netcat（在大多数类Unix系统中找到的小实用程序）作为数据服务器运行

$ nc -lk 9999

$ ./bin/run-example org.apache.spark.examples.sql.streaming.StructuredNetworkWordCount localhost 9999


## 编程模型
结构化流中的关键思想是将实时数据流视为连续追加的表。这导致新的流处理模型非常类似于批处理模型。您将流式计算表示为静态表上的标准批处理查询，Spark将其作为无界输入表上的增量查询运行。让我们更详细地了解这个模型。

